In [ ]:
# def set_hadoop_config(credentials):
    prefix = "fs.swift.service." + credentials['name'] 
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + ".auth.url", credentials['auth_url']+'/v3/auth/tokens')
    hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
    hconf.set(prefix + ".tenant", credentials['project_id'])
    hconf.set(prefix + ".username", credentials['user_id'])
    hconf.set(prefix + ".password", credentials['password'])
    hconf.setInt(prefix + ".http.port", 8080)
    hconf.set(prefix + ".region", credentials['region'])
    hconf.setBoolean(prefix + ".public", True)

In [2]:
credentials = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_42fa9223_5435_461f_8dd5_0cb039c647e6',
  'project_id':'9b56b688275a4bc4b5504219a3d46e83',
  'region':'dallas',
  'user_id':'1b4dfd29b87c437ea36961228e2f9a59',
  'domain_id':'4083817df0394a2a913a8529fd580d22',
  'domain_name':'1138005',
  'username':'admin_16484d02d9ac61b305ea1ea5410b70aeb2bd584e',
  'password':"""Yp^trGUgn4}(j60U""",
  'filename':'ratings_Digital_Music.csv',
  'container':'notebooks',
  'tenantId':'s56d-f5f345bc4fd0b3-4f64990c7b62'
}

In [3]:
credentials['name'] = 'keystone'
set_hadoop_config(credentials)

In [6]:
music = sc.textFile("swift://notebooks.keystone/ratings_Digital_Music.csv")

In [7]:
print "Total records in the Amazon digital music dataset:", music.count()

Total records in the Amazon digital music dataset: 836007


In [10]:
print "The first row in the 2015.csv dataset:", music.first()

The first row in the 2015.csv dataset: USER,SONG,Rating,TIMESTAMP


In [11]:
musicParse = music.map(lambda line : line.split(","))

In [12]:
musicParse.first()

[u'USER', u'SONG', u'Rating', u'TIMESTAMP']

In [13]:
fullRatedMusic = musicParse.filter(lambda x: x[2] == "5")

In [14]:
print fullRatedMusic.count()

622450


In [15]:
# x[1] is the song
# x[2] is the rating
fullRatedMusicByKey = fullRatedMusic.map(lambda x : (x[1], (int(x[2]), 1)))

In [16]:
fullRatedMusicByKey.first()

(u'5555991584', (5, 1))

In [17]:
fullRatedMusicAddByKey = fullRatedMusicByKey.reduceByKey(lambda v1,v2 : (v1[0]+v2[0], v1[1]+v2[1]))

In [18]:
fullRatedMusicAddByKey.first()

(u'B000038ICO', (50, 10))

In [19]:
fullRatedAverages = fullRatedMusicAddByKey.map(lambda k: (k[0], k[1][0] / float(k[1][1])))

In [22]:
fullRatedAverages.first()

(u'B000038ICO', 5.0)

In [23]:
for pair in fullRatedAverages.top(10):
    print "Songs %s had average Rating of %f" % (pair[0],pair[1])

Songs B00LWJDZHI had average Rating of 5.000000
Songs B00LWCRAQ2 had average Rating of 5.000000
Songs B00LTL4VGI had average Rating of 5.000000
Songs B00LTFDZXE had average Rating of 5.000000
Songs B00LQUW3BC had average Rating of 5.000000
Songs B00LQLPTXK had average Rating of 5.000000
Songs B00LQ92TE4 had average Rating of 5.000000
Songs B00LQ92A20 had average Rating of 5.000000
Songs B00LQ8DG24 had average Rating of 5.000000
Songs B00LPXAGV4 had average Rating of 5.000000
